In [1]:
%pylab inline
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import gluonnlp as nlp
import numpy as np
from torch.utils.data import Dataset, DataLoader
from kogpt2.pytorch_kogpt2 import get_pytorch_kogpt2_model
from gluonnlp.data import SentencepieceTokenizer
from kogpt2.utils import get_tokenizer
from tqdm import tqdm, tqdm_notebook
from transformers import AdamW
from transformers.optimization import WarmupLinearSchedule

tok_path = get_tokenizer()
gpt, vocab = get_pytorch_kogpt2_model()
tok = SentencepieceTokenizer(tok_path,  num_best=0, alpha=0)

Populating the interactive namespace from numpy and matplotlib
using cached model
using cached model
using cached model


In [2]:
import random as rd

In [6]:
#device = torch.device("cuda:0")
device = torch.device("cpu")

In [3]:
dataset = pd.read_csv('C:/Users/mskim3/Desktop/식단인식_알고리즘/data/fe_test2.csv',encoding='cp949').iloc[:,1:]

In [4]:
class GPT2summarizer(nn.Module):
    def __init__(self, gpt):
        super(GPT2summarizer, self).__init__()
        self.gpt = gpt

    def forward(self, ids):
        out = self.gpt(ids)

        return out

In [7]:
model = GPT2summarizer(gpt).to(device)
model.load_state_dict(torch.load('C:/Users/mskim3/Desktop/식단인식_알고리즘/model/GPT2_Food_amt.pth', map_location=device))
model.eval()

GPT2summarizer(
  (gpt): GPT2LMHeadModel(
    (transformer): GPT2Model(
      (wte): Embedding(50000, 768)
      (wpe): Embedding(1024, 768)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0): Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): MLP(
            (c_fc): Conv1D()
            (c_proj): Conv1D()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (1): Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0.1, inplace=False)
          

In [8]:
def exampler():
    sent = dataset['sent'].iloc[rd.choice(range(6250))] 
    toked = tok(sent)
    linp = [vocab[vocab.bos_token],]  + vocab[toked] + [vocab[vocab.eos_token]] + [vocab['<sys>']]
    inp = torch.tensor(linp)
    i=0
    while i < 50:
        pred = model(inp)[0]
        predi = torch.argmax(pred[-1]).item()
        if predi == vocab[vocab.eos_token]:
            break
        linp = linp + [predi]
        inp = torch.tensor(linp)
    #finalsent = vocab.to_tokens(linp)
    foodlist = []
    amtlist = []
    timelist = []
    for i in range(len(np.where(np.array(linp)==7)[0])):
        si = np.where(np.array(linp)==7)[0][i]
        ei = np.where(np.array(linp)==8)[0][i]
        foodlist.append(''.join(vocab.to_tokens(linp[si+1:ei])).replace('▁',' ').strip())
    for i in range(len(np.where(np.array(linp)==9)[0])):
        si = np.where(np.array(linp)==9)[0][i]
        ei = np.where(np.array(linp)==10)[0][i]
        amtlist.append(''.join(vocab.to_tokens(linp[si+1:ei])).replace('▁',' ').strip())
    for i in range(len(np.where(np.array(linp)==11)[0])):
        si = np.where(np.array(linp)==11)[0][i]
        ei = np.where(np.array(linp)==12)[0][i]
        timelist.append(''.join(vocab.to_tokens(linp[si+1:ei])).replace('▁',' ').strip())
    print('입력: ', sent)
    print('먹은 음식: ', foodlist)
    print('양: ', amtlist)
    print('시기: ', timelist)
    #print('실제 Output:')
    #print(''.join(vocab.to_tokens(linp)).replace('▁',' ').strip())

In [9]:
def foodextractor(sent):
    toked = tok(sent)
    linp = [vocab[vocab.bos_token],]  + vocab[toked] + [vocab[vocab.eos_token]] + [vocab['<sys>']]
    inp = torch.tensor(linp)
    i=0
    while i < 50:
        pred = model(inp)[0]
        predi = torch.argmax(pred[-1]).item()
        if predi == vocab[vocab.eos_token]:
            break
        linp = linp + [predi]
        inp = torch.tensor(linp)
    #finalsent = vocab.to_tokens(linp)
    foodlist = []
    amtlist = []
    timelist = []
    for i in range(len(np.where(np.array(linp)==7)[0])):
        si = np.where(np.array(linp)==7)[0][i]
        ei = np.where(np.array(linp)==8)[0][i]
        foodlist.append(''.join(vocab.to_tokens(linp[si+1:ei])).replace('▁',' ').strip())
    for i in range(len(np.where(np.array(linp)==9)[0])):
        si = np.where(np.array(linp)==9)[0][i]
        ei = np.where(np.array(linp)==10)[0][i]
        amtlist.append(''.join(vocab.to_tokens(linp[si+1:ei])).replace('▁',' ').strip())
    for i in range(len(np.where(np.array(linp)==11)[0])):
        si = np.where(np.array(linp)==11)[0][i]
        ei = np.where(np.array(linp)==12)[0][i]
        timelist.append(''.join(vocab.to_tokens(linp[si+1:ei])).replace('▁',' ').strip())
    print('입력: ', sent)
    print('먹은 음식: ', foodlist)
    print('양: ', amtlist)
    print('시기: ', timelist)
    #print('실제 Output:')
    #print(''.join(vocab.to_tokens(linp)).replace('▁',' ').strip())

In [10]:
exampler()

입력:  내일 식사는 점심으로 어복쟁반 다섯 조각, 총각 김치 반 조각을 쪼금 먹음
먹은 음식:  ['어복쟁반', '총각 김치']
양:  ['다섯 조각', '반 조각']
시기:  ['점심']


In [61]:
sent = input()
foodextractor(sent)

오늘 3시 30분에 김치 두조각과 밥 한공기를 먹었음
입력:  오늘 3시 30분에 김치 두조각과 밥 한공기를 먹었음
먹은 음식:  ['김치', '밥']
양:  ['두 조각', '한 공기']
시기:  ['간식']


In [ ]:
#debugging

In [ ]:
sent = '오늘 간식으로 부침개 하나 먹었다'
toked = tok(sent)
linp = [vocab[vocab.bos_token],]  + vocab[toked] + [vocab[vocab.eos_token]] + [vocab['<sys>']]
inp = torch.tensor(linp)
i=0
while i < 50:
    pred = model(inp)[0]
    predi = torch.argmax(pred[-1]).item()
    if predi == vocab[vocab.eos_token]:
        break
    linp = linp + [predi]
    inp = torch.tensor(linp)
#finalsent = vocab.to_tokens(linp)
foodlist = []
amtlist = []
timelist = []
for i in range(len(np.where(np.array(linp)==7)[0])):
    si = np.where(np.array(linp)==7)[0][i]
    ei = np.where(np.array(linp)==8)[0][i]
    foodlist.append(''.join(vocab.to_tokens(linp[si+1:ei])).replace('▁',' ').strip())
for i in range(len(np.where(np.array(linp)==9)[0])):
    si = np.where(np.array(linp)==9)[0][i]
    ei = np.where(np.array(linp)==10)[0][i]
    amtlist.append(''.join(vocab.to_tokens(linp[si+1:ei])).replace('▁',' ').strip())
for i in range(len(np.where(np.array(linp)==11)[0])):
    si = np.where(np.array(linp)==11)[0][i]
    ei = np.where(np.array(linp)==12)[0][i]
    timelist.append(''.join(vocab.to_tokens(linp[si+1:ei])).replace('▁',' ').strip())
print('입력: ', sent)
print('먹은 음식: ', foodlist)
print('양: ', amtlist)
print('시기: ', timelist)

In [115]:
vocab.to_tokens(linp)

['<s>',
 '▁오늘',
 '▁간',
 '식으로',
 '▁부',
 '침',
 '개',
 '▁하나',
 '▁먹었다',
 '</s>',
 '<sys>',
 '<unused5>',
 '▁간식',
 '<unused6>',
 '<unused1>',
 '▁부',
 '침',
 '개',
 '<unused2>',
 '<unused3>',
 '▁하나',
 '<unused2>',
 '<unused3>',
 '▁N',
 'one',
 '<unused4>']

In [101]:
import konlpy
from konlpy.tag import Okt

In [102]:
okt = Okt()

In [103]:
inp = input()
okt.pos(inp)

오늘 점심으로 제육볶음이랑 밥 한공기 먹었어


[('오늘', 'Noun'),
 ('점심', 'Noun'),
 ('으로', 'Josa'),
 ('제', 'Modifier'),
 ('육', 'Modifier'),
 ('볶음', 'Noun'),
 ('이랑', 'Josa'),
 ('밥', 'Noun'),
 ('한', 'Determiner'),
 ('공기', 'Noun'),
 ('먹었어', 'Verb')]